In [4]:

#from google.colab import drive
#drive.mount('/content/gdrive')
#import os
#path = "/content/gdrive/MyDrive/noncomplaint_detect"

#os.chdir(path)
#os.listdir(path)

Mounted at /content/gdrive


['noncomplaint.txt', 'complaint.txt', 'customertweets.csv', 'final(1).ipynb']

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Embedding, Dropout, TextVectorization, Bidirectional, LSTM, GlobalMaxPooling1D
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
import re

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [23]:
import tensorflow as tf
tf.test.gpu_device_name()

2024-03-07 20:48:19.617449: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-07 20:48:19.617504: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


'/device:GPU:0'

In [3]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
max_sequence_length = 600
vocab_size = 100000
max_num_word = 1000000


In [4]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [5]:
def load_and_preprocess_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().split('\n')
    cleaned_lines = [clean_text(line) for line in lines if line]
    return cleaned_lines


complaints_data = load_and_preprocess_data('/Users/emmali/Downloads/complaint.txt')
non_complaints_data = load_and_preprocess_data('/Users/emmali/Downloads/noncomplaint.txt')

complaints_data = file.read('/Users/emmali/Downloads/complaint.txt')
non_complaints_data = file.read('/Users/emmali/Downloads/noncomplaint.txt')


In [6]:
texts = complaints_data + non_complaints_data
labels = [0] * len(complaints_data) + [1] * len(non_complaints_data)

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [7]:
tokenizer = Tokenizer(num_words=max_num_word)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
X_test = pad_sequences(sequences_test, maxlen=max_sequence_length)



In [8]:
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)


2024-03-07 20:13:55.019039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-07 20:13:55.019063: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
#new
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)


In [10]:
def build_model(vocab_size, embedding_dim, input_length):
    inputs = Input(shape=(input_length,))
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length)(inputs)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Bidirectional(LSTM(16))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)
    return model


In [12]:
model_1 = build_model(vocab_size, 256, max_sequence_length)
model_1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint('model_checkpoint_path.h5', save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=3)
]

model_1.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_split=0.2, callbacks=callbacks)

test_loss, test_acc = model_1.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.3f}')

Epoch 1/20


2024-03-07 20:07:44.303581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


312/312 [==============================] - 58s 176ms/step - loss: 0.4326 - accuracy: 0.8086 - val_loss: 0.3332 - val_accuracy: 0.8617
Epoch 2/20


/Users/emmali/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


312/312 [==============================] - 52s 167ms/step - loss: 0.2753 - accuracy: 0.8941 - val_loss: 0.3259 - val_accuracy: 0.8705
Epoch 3/20
312/312 [==============================] - 51s 165ms/step - loss: 0.2177 - accuracy: 0.9180 - val_loss: 0.3323 - val_accuracy: 0.8657
Epoch 4/20
312/312 [==============================] - 52s 168ms/step - loss: 0.1809 - accuracy: 0.9342 - val_loss: 0.3703 - val_accuracy: 0.8585
Epoch 5/20
98/98 [==============================] - 5s 55ms/step - loss: 0.4219 - accuracy: 0.8380
Test Accuracy: 0.838


In [11]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Divide the data set into training set, validation set and test set
X_train, X_val_test, y_train, y_val_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Create a TextVectorization instance and fit it to the training set to build the vocabulary
max_tokens = vocab_size
output_mode = 'int'
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_tokens,output_sequence_length=max_sequence_length, output_mode=output_mode)
text_vectorizer.adapt(X_train)

# Define a function to vectorize text and labels
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return text_vectorizer(text), label

# Use tf.data.Dataset.from_tensor_slices to divide the data set into batches, and call the vectorize_text function through the map function to vectorize the text
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
train_dataset = train_dataset.map(vectorize_text)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
val_dataset = val_dataset.map(vectorize_text)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)
test_dataset = test_dataset.map(vectorize_text)

# Process the data set to improve the efficiency of data loading
train_ds = train_dataset.cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_dataset.cache().prefetch(tf.data.AUTOTUNE)

2024-03-07 20:14:00.538436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [12]:
inputs = keras.Input( shape=(None,), dtype="int64" )
d_emb = 256

In [13]:
embedded = Embedding( input_dim=vocab_size, output_dim=d_emb )(inputs)
x = Bidirectional(LSTM(32, return_sequences=True))(embedded)
x = Bidirectional(LSTM(16))(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)
model_rnn = keras.Model(inputs, outputs)
model_rnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         25600000  
                                                                 
 bidirectional (Bidirection  (None, None, 64)          73984     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 32)                10368     
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33    

In [14]:
model_rnn.compile( optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"] )
callbacks = [
    ModelCheckpoint("/Users/emmali/Downloads/models/complaints_embeddings_bidir_gru.keras", save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=3)
]
model_rnn.fit( train_ds, validation_data=val_ds, epochs=20, callbacks = callbacks )
print(f"Test acc: {model_rnn.evaluate(test_ds)[1]:.3f}")

Epoch 1/20
390/390 [==============================] - 68s 168ms/step - loss: 0.4203 - accuracy: 0.8165 - val_loss: 0.3695 - val_accuracy: 0.8332
Epoch 2/20
390/390 [==============================] - 61s 158ms/step - loss: 0.2867 - accuracy: 0.8866 - val_loss: 0.3592 - val_accuracy: 0.8467
Epoch 3/20
390/390 [==============================] - 61s 157ms/step - loss: 0.2426 - accuracy: 0.9061 - val_loss: 0.3676 - val_accuracy: 0.8448
Epoch 4/20
390/390 [==============================] - 61s 156ms/step - loss: 0.2069 - accuracy: 0.9231 - val_loss: 0.3756 - val_accuracy: 0.8352
Epoch 5/20
49/49 [==============================] - 2s 50ms/step - loss: 0.3921 - accuracy: 0.8409
Test acc: 0.841


In [26]:
column_names = ['id', 'tweet']
df = pd.read_csv('/Users/emmali/Desktop/Simon/2024 Spring A/AI and Deep Learning (CIS433.31A.SPRINGA2024SIMON) CIS433.31A.SPRINGA2024SIMON/Final/customertweets.csv', header=None, names=column_names)
df

,id,tweet
0,6,My 8:50 flight to Dallas has been delayed to 1...
1,7,@suzij77 @united @jimjefferies @Delta Oh that ...
2,8,@ArianFoster I hate @united 2. They didn't let...
3,9,I'm so upset with @AmericanAir They changed my...
4,10,Slightly delayed flight home from Philly. but ...
...,...,...
157205,173658,@DeltaAssist havoc at gate . Flight prior dela...
157206,173659,@united flight got cancelled. baggage is missi...
157207,173660,@Jetblue I've been on hold 36 minutes. My flig...
157208,173661,Hey @United 60 min wait on customer service li...


In [28]:
df['cleaned_tweet'] = df['tweet'].apply(clean_text)
df

,id,tweet,cleaned_tweet
0,6,My 8:50 flight to Dallas has been delayed to 1...,My 8:50 flight to Dallas has been delayed to 1...
1,7,@suzij77 @united @jimjefferies @Delta Oh that ...,Oh that sucks. Deep breath though. don't take ...
2,8,@ArianFoster I hate @united 2. They didn't let...,I hate 2. They didn't let me on my flight 2 we...
3,9,I'm so upset with @AmericanAir They changed my...,I'm so upset with They changed my boyfriends f...
4,10,Slightly delayed flight home from Philly. but ...,Slightly delayed flight home from Philly. but ...
...,...,...,...
157205,173658,@DeltaAssist havoc at gate . Flight prior dela...,havoc at gate . Flight prior delayed . our fli...
157206,173659,@united flight got cancelled. baggage is missi...,flight got cancelled. baggage is missing and y...
157207,173660,@Jetblue I've been on hold 36 minutes. My flig...,I've been on hold 36 minutes. My flight was ca...
157208,173661,Hey @United 60 min wait on customer service li...,Hey 60 min wait on customer service line. Any ...


In [29]:
from tqdm import tqdm

def vectorize_text(text):
    text = tf.expand_dims(text, -1)
    return text_vectorizer(text)


tqdm.pandas()

df['vectorized_tweet'] = df['cleaned_tweet'].progress_apply(vectorize_text)

100%|██████████| 157210/157210 [46:04<00:00, 56.86it/s]   


In [30]:
vectorized_tweets = df['vectorized_tweet'].apply(lambda x: x[0])

predictions = model_rnn.predict(np.array(vectorized_tweets.tolist()))

4913/4913 [==============================] - 261s 53ms/step


In [31]:
df['prediction_prob'] = predictions
df_sorted = df[df['prediction_prob']>0.5].sort_values('prediction_prob', ascending=False)
df_sorted

,id,tweet,cleaned_tweet,vectorized_tweet,prediction_prob
46770,52299,@JetBlue Can't thank your customer service tea...,Can't thank your customer service team enough....,"((tf.Tensor(17, shape=(), dtype=int64), tf.Ten...",0.996903
112478,124681,#JetBlue ShoutOUT 2 @JetBlue Boston Crew hated...,ShoutOUT 2 Boston Crew hated to leave Florida....,"((tf.Tensor(613, shape=(), dtype=int64), tf.Te...",0.996877
125665,139200,thank you @JetBlue for an amazing experience. ...,thank you for an amazing experience. flight al...,"((tf.Tensor(125, shape=(), dtype=int64), tf.Te...",0.996705
31053,34653,can't wait until this video leaks (it's going ...,can't wait until this video leaks (it's going ...,"((tf.Tensor(17, shape=(), dtype=int64), tf.Ten...",0.996572
126950,140624,@radiodrew @SouthwestAir Heading there after w...,Heading there after work today!!! Can't wait! ...,"((tf.Tensor(660, shape=(), dtype=int64), tf.Te...",0.996559
...,...,...,...,...,...
127329,141028,"@airlinewriter @AlaskaAir Don't buy the ""fell ...","Don't buy the ""fell asleep"" claim--just the ca...","((tf.Tensor(87, shape=(), dtype=int64), tf.Ten...",0.500119
98898,109673,No. this isn't FRA it's IAD. my flt 1st was cx...,No. this isn't FRA it's IAD. my flt 1st was cx...,"((tf.Tensor(24, shape=(), dtype=int64), tf.Ten...",0.500082
34646,38705,@SouthwestAir tring to change internationa iti...,tring to change internationa itinareray usa to...,"((tf.Tensor(1, shape=(), dtype=int64), tf.Tens...",0.500052
115451,127948,I will NEVER fly @AmericanAir again if I get t...,I will NEVER fly again if I get to make the ch...,"((tf.Tensor(4, shape=(), dtype=int64), tf.Tens...",0.500042


In [32]:
df_sorted[['id','tweet']].to_csv('/Users/emmali/Downloads/non_complaint_tweets_sorted.csv',index=None)